# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [3]:
# Your solution goes here
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [5]:
# Mapping of star-rating words to numeric values
RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

BASE_URL = "http://books.toscrape.com/"

def get_genre(book_url):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    breadcrumb = soup.select("ul.breadcrumb li a")
    if len(breadcrumb) >= 3:
        return breadcrumb[2].text.strip()
    return None

def get_description(soup):
    desc = soup.find('meta', attrs={"name": "description"})
    if desc:
        return desc.get("content").strip()
    return "No description available."

def get_upc(soup):
    try:
        return soup.find('th', text='UPC').find_next_sibling('td').text
    except:
        return None

def scrape_books(min_rating=4, max_price=20.0):
    books = []
    page_num = 1

    while True:
        url = f"{BASE_URL}catalogue/page-{page_num}.html"
        response = requests.get(url)
        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.content, 'html.parser')
        book_items = soup.select('article.product_pod')

        for item in book_items:
            # Get rating
            rating_class = item.find('p', class_='star-rating')['class'][1]
            rating = RATING_MAP.get(rating_class, 0)
            if rating < min_rating:
                continue

            # Get price
            price = float(item.find('p', class_='price_color').text[2:])
            if price > max_price:
                continue

            title = item.h3.a['title']
            book_relative_url = item.h3.a['href'].replace('../../../', '')
            book_url = BASE_URL + "catalogue/" + book_relative_url

            book_response = requests.get(book_url)
            book_soup = BeautifulSoup(book_response.content, 'html.parser')

            upc = get_upc(book_soup)
            genre = get_genre(book_url)
            availability = book_soup.find('p', class_='instock availability').text.strip()
            description = get_description(book_soup)

            books.append({
                "UPC": upc,
                "Title": title,
                "Price (£)": price,
                "Rating": rating,
                "Genre": genre,
                "Availability": availability,
                "Description": description
            })

            time.sleep(0.2)  # Be polite to the server

        page_num += 1

    return pd.DataFrame(books)


In [7]:
df = scrape_books(min_rating=4, max_price=20.0)
print(df.head())

/var/folders/0b/0nt_hmld68j1x9vbn6980snc0000gn/T/ipykernel_83009/917534377.py:28: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  return soup.find('th', text='UPC').find_next_sibling('td').text


                UPC                                              Title  \
0  e00eb4fd7b871a48                                      Sharp Objects   
1  4165285e1663650f              Sapiens: A Brief History of Humankind   
2  2597b5a345f45e1b  The Dirty Little Secrets of Getting Your Dream...   
3  e10e1e165dc8be4a  The Boys in the Boat: Nine Americans and Their...   
4  30a7f60cd76ca58c                              Shakespeare's Sonnets   

   Price (£)  Rating     Genre             Availability  \
0       7.82       4   Mystery  In stock (20 available)   
1       4.23       5   History  In stock (20 available)   
2       3.34       4  Business  In stock (19 available)   
3       2.60       4   Default  In stock (19 available)   
4       0.66       4    Poetry  In stock (19 available)   

                                         Description  
0  WICKED above her hipbone, GIRL across her hear...  
1  From a renowned historian comes a groundbreaki...  
2  Drawing on his extensive experie